# Food For Thought 
## Digging into the dataset

The first step of this project aims at understanding the dataset we have chosen ([Open Food Facts Database](https://world.openfoodfacts.org/)), to check whether it is suitable for the kind of analysis we want to develop.

As a reminder, we would like to focus our research on two main topics:
1. Impact of food on environment
2. Impact of food on the health

In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.style as style
from mpl_toolkits.mplot3d import Axes3D
from difflib import get_close_matches

import pickle

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import findspark
findspark.init()

from pyspark.sql import *
import pyspark.sql.functions as F

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
%matplotlib inline

In [2]:
from plotly.offline import download_plotlyjs, init_notebook_mode
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tools

tools.set_credentials_file(username='tomM', api_key='6IDmi0n3tVu5rfMzN9DZ')
init_notebook_mode(connected=True)

## load the raw csv file

## Brand vs Nutrition vs Food Category in France 

## load and clean

In [ ]:
# we can now read it everytime we want to continue the analysis
bra_cat_nut_raw_df = pd.read_pickle('bra_cat_nut_raw_df')

In [ ]:
bra_cat_nut_raw_df.head(3)

- We create a dictonary of parent brand to replace a brand by its "parent brand" when it belongs to a bigger one
For example 'Bio Village' is a brand of Leclerc which owes also 'Marque Repère' then we can replace 'Bio Village' and 'Marque Repère' by 'Leclerc'. We will to analysis the food sold by the company Leclerc.


- Moreover, we set to None the brand when it is "Sans Marque". "Sans Marque" means the brand had not been specified by the user: it can be "Marque Repère", "Carrefour", "Délisse" ect.To understand the 'Sans Marque' products we checked them using the product search tool on https://fr.openfoodfacts.org with the tag "Sans Marque".

In [ ]:
# list of food brands present in France ('Coca Cola', 'Fleury Michon', 'Carrefour' ect.)
list_brand_file = open("brand_list.txt", "r")
list_brand = list_brand_file.read().split('\n')

# dictonary of parent brands. 
dict_parent_brand= {'Bio Village': 'Leclerc',
                    'Marque Repère': 'Leclerc',
                    'Sans Marque': None}

The problem is that the 'brands' tags are not always consistent. For example we can find the brand tag 'carrefour', "Carrefour", "carrfour,bio carrefour' ect. 
Hence we first clean the brands using a list of food brands present in France and a function to match the 'brands' tag with the closest brand in the list. 

In [ ]:
# Function which finds for each value (string) in the column 'brands' the closest brand (string) in 'list_brand'.
# Note: the function 'get_close_matches' has been imported from the open-source 'difflib' library.
def clean_brand(brand_name):
    
    # brand name can be of the form 'Carrefour' (one word) or 'Carrefour, Bio Carrefour' (multiple words)
    # hence we split the string w.r.t the symbol ',' and find the best matching brand word for each split.
    matches=[]
    if brand_name:
        
        for word in brand_name.split(','):

            # the cutoff control for the False Positives possibilities in the 'list_brand' 
            # that don’t score at least the cutoff are ignored.
            # the parameter 'n' controls for the the number of possibilities (whose score is higher than the cutoff value).
            matches.append(get_close_matches(word.lower(), list_brand, n=1, cutoff=0.6))

        # remove empty sublists and unravel
        matches = [brand for sublist in matches for brand in sublist if sublist]
        
        # output brand
        if matches:
            output = matches[0]
        else:
            output = None
                
        # check for parent brand
        if output in dict_parent_brand:
            output = dict_parent_brand[output]
        
        return output

In [ ]:
# we create a new column with the new "consistent" brand tag. 
# takes some minutes to run...
bra_cat_nut_raw_df['new_brands'] = bra_cat_nut_raw_df['brands'].apply(lambda x: clean_brand(x))

In [ ]:
# drop when both 'nutrition_grade_fr' and 'brands' are None
bra_cat_nut_cleaned_df = bra_cat_nut_raw_df.dropna(subset=['nutrition_grade_fr', 'brands'], how='all') 

# we can drop the old column 'brands' since we have the new ones.
bra_cat_nut_cleaned_df = bra_cat_nut_cleaned_df.drop('brands',1).rename(columns={'new_brands':'brands'})

bra_cat_nut_cleaned_df.head()

In [ ]:
# here we count the number of products for each brand. 
count = bra_cat_nut_cleaned_df.groupby('brands').count().sort_values('nutrition_grade_fr', ascending=False)

In [ ]:
# we consider that brand can be analysed if at least 100 products in the dataset belong to this brand.
count['enough_products']= count[['nutrition_grade_fr']].apply(lambda x: x>100)
count = count[count.enough_products]
count.head()

The most abundant brands are Carrefour, Auchan and U. There are the brands of the biggest distributors (supermarkets) in France. 

In [ ]:
# here we add the column "enough products" to the dataframe with the cleaned brand.
bra_cat_nut_cleaned_df = bra_cat_nut_cleaned_df.join(count[['enough_products']], on='brands')
bra_cat_nut_cleaned_df = bra_cat_nut_cleaned_df.drop('enough_products',1)
bra_cat_nut_cleaned_df.head()

The nutrition score in 'nutrition_grade_fr' is the Nutri-Score developped by the french governement and based on the components present in the food (sugar, fiber, fat ect.). You can find more information at https://fr.wikipedia.org/wiki/Nutri-score.

- 'a' (very good product for health) 
- 'b' (good product)
- 'c' ('neutral product')
- 'd' ('not so good product')
- 'e' (bad product for health)

In [ ]:
# here we create dummy variables from the 'nutrition_grade_fr'.
bra_cat_nut_cleaned_expanded_df = pd.get_dummies(bra_cat_nut_cleaned_df.set_index('brands')).reset_index()
bra_cat_nut_cleaned_dummies_df = bra_cat_nut_cleaned_expanded_df.rename(columns=
                              {'nutrition_grade_fr_a':'a',
                               'nutrition_grade_fr_b':'b',
                               'nutrition_grade_fr_c':'c',
                               'nutrition_grade_fr_d':'d',
                               'nutrition_grade_fr_e':'e',
                               'pnns_groups_1_Beverages': 'Beverages',
                               'pnns_groups_1_Cereals And Potatoes':'Cereals And Potatoes',
                               'pnns_groups_1_Composite Foods': 'Composite Foods',
                               'pnns_groups_1_Fat And Sauces':'Fat And Sauces',
                               'pnns_groups_1_Fish Meat Eggs':'Fish Meat Eggs',
                               'pnns_groups_1_Fruits And Vegetables':'Fruits And Vegetables',
                               'pnns_groups_1_Milk And Dairy Products':'Milk And Dairy Products',
                               'pnns_groups_1_Salty Snacks':'Salty Snacks',
                               'pnns_groups_1_Sugary Snacks':'Sugary Snacks',
                               'pnns_groups_1_Unknown': 'Unknown'})
bra_cat_nut_cleaned_dummies_df.head(5)

## Load cleaned data

In [3]:
bra_cat_nut_df = pd.read_pickle('bra_cat_nut_cleaned_df')
bra_cat_nut_dummies_df = pd.read_pickle('bra_cat_nut_cleaned_dummies_df')

## Food quality in France

In [4]:
# here we count the number of products inside each nutrition category (5 categories from 'a' to 'e')
count = pd.DataFrame(data=bra_cat_nut_dummies_df[['a','b','c','d','e']].sum(), columns=['count'])
count.index.name = 'nutrition_grade_fr'

In [5]:
# we convert count into ratio (regarding the total number of products)
number_of_products = bra_cat_nut_df.nutrition_grade_fr.count()
print("Total number of products sold in France:", number_of_products)

ratio = count.div(number_of_products).multiply(100).rename(columns={'count':'ratio'})
ratio.head(3)

Total number of products sold in France: 103571


,ratio
nutrition_grade_fr,
a,13.111778
b,13.764471
c,20.704637


In [1]:
# interactive plot
data = []
colors = sns.color_palette("RdBu_r", 5).as_hex()

for index, row in ratio.reset_index().iterrows():
    data.append(go.Bar(x=np.array(index),
                       y=np.array(row['ratio']),
                       name = row['nutrition_grade_fr'],
                       marker={'color': colors[index]}))

layout = go.Layout(
    title='Nutri-Score repartition in France.',
    xaxis=go.layout.XAxis(
        title='Nutriscore',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        tickvals=np.arange(5).astype(str),
        ticktext=ratio.index,
    ),
    yaxis=dict(
        title='Ratio (in %)',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
)

fig = go.FigureWidget(data=data, layout=layout)
fig

NameError: name 'sns' is not defined

In [7]:
py.plot(fig, filename='food_quality_in_france_bar', auto_open=False)

'https://plot.ly/~tomM/94'

In [8]:
# interactive plot
labels = count.index
values = count['count'].values
colors = sns.color_palette("RdBu_r", 5).as_hex()

fig = {
    'data': [{'labels': labels,
              'values': values,
              'type': 'pie',
              'marker':{'colors':colors}
              }
            ],
    'layout': {'title': 'Nutri-Score repartition in France.'}
     }

py.iplot(fig, filename='food_quality_in_france_pie', auto_open=False)

We can observe than, unfortunalty, more than 50% of the products sold in France have a bad nutrition grade (d or e). 
The healthy product tagged with the grade 'a' only represent 13% of the ~100,000 products in the data set.

## Food quality for each french food brand

In [9]:
# read the data
bra_nut_df = bra_cat_nut_df[['brands', 'nutrition_grade_fr']]
bra_nut_dummies_df = bra_cat_nut_dummies_df[['brands','a','b','c','d','e']]

In [10]:
# extract the total number of products for each brands
brand_count = bra_nut_df.groupby('brands').count().rename(columns={'nutrition_grade_fr':'total'})

In [11]:
# count the nutrition grade occurences for each brand
brand_dummies_count = bra_nut_dummies_df.groupby('brands').sum()

In [12]:
# we add the column 'total' to further extract the ratio of each grade instead of the raw count.
brand_dummies_count = brand_dummies_count.join(brand_count)
brand_dummies_count.head()

,a,b,c,d,e,total
brands,,,,,,
123 bio,121.0,75.0,54.0,91.0,80.0,421
7Up,0.0,5.0,1.0,4.0,24.0,34
A l'olivier,0.0,2.0,19.0,43.0,3.0,67
Ajax,1.0,0.0,0.0,0.0,1.0,2
Albert Menes,2.0,6.0,27.0,37.0,22.0,94


In [13]:
# we convert the counts into ratios. 
# ex: ratio for 'a' = (count for 'a') / (total number of products)
brand_nutri_ratio = brand_dummies_count.copy()
brand_nutri_ratio[['a','b','c','d','e']] = brand_dummies_count[['a','b','c','d','e']].div(brand_dummies_count['total'].values,axis=0)*100
brand_nutri_ratio = brand_nutri_ratio.sort_values('total',ascending=False)

In [14]:
# extract the 10 biggest brands
ratio_top10 = brand_nutri_ratio.reset_index().loc[:9].drop('total',1)

# save the brand names for further analysis 
top10_brand_names = ratio_top10.brands.values

ratio_top10.head()

,brands,a,b,c,d,e
0,Carrefour,19.763931,15.344496,19.818831,26.269558,18.803184
1,Auchan,18.391764,15.303283,21.313300,26.794658,18.196995
2,Leclerc,13.648180,15.901213,22.010399,27.816291,20.623917
3,Casino,18.849840,15.609311,20.903697,25.604747,19.032405
4,Leader Price,17.482517,13.939394,22.750583,26.293706,19.533800


In [15]:
# interactive plot
data = []
colors = sns.color_palette("tab10", ratio_top10.brands.count()).as_hex()

for index, row in ratio_top10.iterrows():
    data.append(go.Bar(x=['a','b','c','d','e'],
                       y=np.array(row[1:]),
                       name = row['brands'],
                       marker={'color': colors[index]}))

layout = go.Layout(
    title='Nutrition score occurences for the 10 biggest food brands in France.',
    xaxis=go.layout.XAxis(
        title='Nutriscore',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        tickvals=np.arange(5).astype(str),
        ticktext=['a','b','c','d','e'],
    ),
    yaxis=dict(
        title='Ratio (in %)',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
)

fig = go.FigureWidget(data=data, layout=layout)
py.iplot(fig, filename='brands_vs_nutriscore_bar', auto_open=False)

In [16]:
# interactive plot
data = []
colors = sns.color_palette("RdBu_r", 5).as_hex()

for index in range(len(['a','b','c','d','e'])):
    data.append(go.Bar(x=ratio_top10['brands'],
                       y=ratio_top10[ratio_top10.columns[index+1]],
                       name = ratio_top10.columns[index+1],
                       marker={'color': colors[index]}))

layout = go.Layout(
    
    title='Nutrition score distribution within the 10 biggest food brands in France.',
    barmode='stack',
    
    # x axis
    xaxis=go.layout.XAxis(
        title='Food Brand',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        tickvals=np.arange(10).astype(str),
        ticktext=ratio_top10['brands'],
    ),
    
    # y axis
    yaxis=dict(
        title='Ratio (in %)',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        showgrid=True,
        gridcolor='#bdbdbd',
    ),
)

fig = go.FigureWidget(data=data, layout=layout)
py.iplot(fig, filename='brands_vs_nutriscore_bar_stacked', auto_open=False)

As seen previously, we observe that for almost all the biggest food brands present in France more than 40% of the products are "junk" food (products with labels 'e' or 'd'). We don't eat that well in France... 

The exception is 'Picard' in which products tagged with 'd' or 'e' represent one third of the total number of products (which is already quite high...). Actually, Picard mainly sells frozen food which could be excpected to be healthier than the products sold by Netto (cheap product mainly industrial).

## Category Distribution in the 10 biggest food brands in France. 

In [17]:
# read the data
bra_cat_df = bra_cat_nut_df[['brands', 'pnns_groups_1']]
bra_cat_dummies_df = bra_cat_nut_dummies_df.drop(['a','b','c','d','e'], axis=1)

In [18]:
# here we count the category occurences for each brand
brand_dummies_count = bra_cat_dummies_df.groupby('brands').sum()

# we add the column total to further extract the ratio of each grade instead of the raw count
brand_count = pd.DataFrame(data=brand_dummies_count.sum(axis=1), columns=['total'])
brand_dummies_count = brand_dummies_count.join(brand_count)

brand_dummies_count.head(3)

,Beverages,Cereals And Potatoes,Composite Foods,Fat And Sauces,Fish Meat Eggs,Fruits And Vegetables,Milk And Dairy Products,Salty Snacks,Sugary Snacks,Unknown,total
brands,,,,,,,,,,,
123 bio,38.0,60.0,28.0,38.0,28.0,64.0,43.0,15.0,72.0,222.0,608.0
7Up,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,37.0
A l'olivier,0.0,1.0,2.0,41.0,1.0,0.0,0.0,0.0,2.0,67.0,114.0


In [19]:
# we focus on the 10 biggest brands
ratio_bra_cat = brand_dummies_count.copy().reset_index()
ratio_bra_cat = ratio_bra_cat[ratio_bra_cat['brands'].isin(top10_brand_names)].set_index('brands')

In [20]:
# we convert the counts into ratios
ratio_bra_cat.iloc[:, np.arange(10)] = brand_dummies_count.iloc[:,np.arange(10)].div(brand_dummies_count['total'].values,axis=0)*100

In [21]:
ratio_bra_cat.sort_values(by='Unknown')

,Beverages,Cereals And Potatoes,Composite Foods,Fat And Sauces,Fish Meat Eggs,Fruits And Vegetables,Milk And Dairy Products,Salty Snacks,Sugary Snacks,Unknown,total
brands,,,,,,,,,,,
Belle France,6.370795,7.229778,11.739442,5.941303,9.949893,9.305655,13.815319,4.509664,15.175376,15.962777,1397.0
Picard,0.539568,2.637890,37.170264,1.738609,9.472422,10.071942,8.932854,2.218225,8.992806,18.225420,1668.0
Monoprix,5.847953,8.878256,11.483254,4.731526,9.941520,7.921318,9.994684,3.030303,12.706007,25.465178,1881.0
Leader Price,5.774783,6.256015,11.004171,4.202759,8.020533,6.127687,10.137953,2.983638,10.619185,34.873276,3117.0
Carrefour,4.881403,8.078377,8.525266,4.331385,8.765899,7.356480,7.751805,2.543829,10.244070,37.521485,5818.0
Leclerc,6.940737,7.234383,8.622531,2.162306,8.649226,5.125467,7.020822,1.067806,14.281901,38.894821,3746.0
Casino,6.231714,7.050907,9.215916,3.832651,6.495026,6.143944,7.665301,2.837917,10.269163,40.257461,3418.0
Auchan,5.265011,6.374362,8.681106,4.824793,8.188061,6.691319,8.874802,2.201092,7.941539,40.957915,5679.0
Cora,6.791467,4.658250,7.792773,4.309970,8.663474,6.704397,6.269047,3.439269,10.317806,41.053548,2297.0


We chose to remove the category Unknown since it gathers products from multiple categories. It is not relevant to analyze the distribution of products tagged 'Unknown' since they don't represent any real category of products. The user just omitted to tag the category. 

In [22]:
ratio_to_plot = ratio_bra_cat.reset_index().drop(['total', 'Unknown'],1)

In [23]:
# interactive plot
data = []
colors = sns.color_palette("tab10", ratio_to_plot.brands.count()).as_hex()

for index, nutri in enumerate(ratio_to_plot.columns[1:]):
    data.append(go.Bar(x=ratio_to_plot['brands'],
                       y=ratio_to_plot[ratio_to_plot.columns[index+1]],
                       name = ratio_to_plot.columns[index+1],
                       marker={'color': colors[index]}))

layout = go.Layout(
    title='Food Category distribution within the 10 biggest food brands in France.',
    xaxis=go.layout.XAxis(
        title='Food Brand',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        tickvals=np.arange(10).astype(str),
        ticktext=ratio_to_plot['brands'],
    ),
    yaxis=dict(
        title='Ratio (in %)',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
)

go.FigureWidget(data=data, layout=layout)

FigureWidget({
    'data': [{'marker': {'color': '#1f77b4'},
              'name': 'Beverages',
              'type': 'bar',
              'uid': 'd645526c-011c-11e9-94bc-ac220bb8d9d3',
              'x': array(['Auchan', 'Belle France', 'Carrefour', 'Casino', 'Cora', 'Leader Price',
                          'Leclerc', 'Monoprix', 'Netto', 'Picard'], dtype=object),
              'y': array([5.26501145, 6.37079456, 4.88140254, 6.23171445, 6.79146713, 5.77478345,
                          6.94073679, 5.84795322, 3.866171  , 0.53956835])},
             {'marker': {'color': '#ff7f0e'},
              'name': 'Cereals And Potatoes',
              'type': 'bar',
              'uid': 'd645526d-011c-11e9-b793-ac220bb8d9d3',
              'x': array(['Auchan', 'Belle France', 'Carrefour', 'Casino', 'Cora', 'Leader Price',
                          'Leclerc', 'Monoprix', 'Netto', 'Picard'], dtype=object),
              'y': array([6.37436168, 7.2297781 , 8.07837745, 7.05090696, 4.65824989, 6.2560154 ,
                          7.23438334, 8.87825625, 5.05576208, 2.63788969])},
             {'marker': {'color': '#2ca02c'},
              'name': 'Composite Foods',
              'type': 'bar',
              'uid': 'd645526e-011c-11e9-b744-ac220bb8d9d3',
              'x': array(['Auchan', 'Belle France', 'Carrefour', 'Casino', 'Cora', 'Leader Price',
                          'Leclerc', 'Monoprix', 'Netto', 'Picard'], dtype=object),
              'y': array([ 8.68110583, 11.73944166,  8.52526641,  9.21591574,  7.79277318,
                          11.00417068,  8.6225307 , 11.48325359,  8.92193309, 37.17026379])},
             {'marker': {'color': '#d62728'},
              'name': 'Fat And Sauces',
              'type': 'bar',
              'uid': 'd645526f-011c-11e9-b659-ac220bb8d9d3',
              'x': array(['Auchan', 'Belle France', 'Carrefour', 'Casino', 'Cora', 'Leader Price',
                          'Leclerc', 'Monoprix', 'Netto', 'Picard'], dtype=object),
              'y': array([4.8247931 , 5.94130279, 4.33138536, 3.83265067, 4.30996953, 4.20275906,
                          2.16230646, 4.73152578, 4.16356877, 1.73860911])},
             {'marker': {'color': '#9467bd'},
              'name': 'Fish Meat Eggs',
              'type': 'bar',
              'uid': 'd6455270-011c-11e9-b6e7-ac220bb8d9d3',
              'x': array(['Auchan', 'Belle France', 'Carrefour', 'Casino', 'Cora', 'Leader Price',
                          'Leclerc', 'Monoprix', 'Netto', 'Picard'], dtype=object),
              'y': array([8.18806128, 9.94989263, 8.76589893, 6.49502633, 8.6634741 , 8.02053256,
                          8.64922584, 9.94152047, 7.50929368, 9.47242206])},
             {'marker': {'color': '#8c564b'},
              'name': 'Fruits And Vegetables',
              'type': 'bar',
              'uid': 'd6455271-011c-11e9-aca8-ac220bb8d9d3',
              'x': array(['Auchan', 'Belle France', 'Carrefour', 'Casino', 'Cora', 'Leader Price',
                          'Leclerc', 'Monoprix', 'Netto', 'Picard'], dtype=object),
              'y': array([ 6.69131889,  9.30565497,  7.35647989,  6.14394383,  6.70439704,
                           6.12768688,  5.12546716,  7.92131845,  5.27881041, 10.07194245])},
             {'marker': {'color': '#e377c2'},
              'name': 'Milk And Dairy Products',
              'type': 'bar',
              'uid': 'd6455272-011c-11e9-9330-ac220bb8d9d3',
              'x': array(['Auchan', 'Belle France', 'Carrefour', 'Casino', 'Cora', 'Leader Price',
                          'Leclerc', 'Monoprix', 'Netto', 'Picard'], dtype=object),
              'y': array([ 8.8748019 , 13.81531854,  7.75180474,  7.66530135,  6.26904658,
                          10.13795316,  7.02082221,  9.99468368,  8.2527881 ,  8.93285372])},
             {'marker': {'color': '#7f7f7f'},
              'name': 'Salty Snacks',
              'type': 'bar',
              'uid': 'd6455273-011c-11e9-8995-ac220bb8d9d3',
              'x': array(

In [24]:
# interactive plot
data = []
colors = sns.color_palette("tab10", len(ratio_to_plot.columns[1:])).as_hex()

for index, nutri in enumerate(ratio_to_plot.columns[1:]):
    data.append(go.Scatter(x=ratio_to_plot['brands'],
                       y=ratio_to_plot[ratio_to_plot.columns[index+1]],
                       name = ratio_to_plot.columns[index+1],
                       marker={'color': colors[index]}
                          )
               )
    
fig = tools.make_subplots(rows=3, cols=3, subplot_titles=tuple(ratio_to_plot.columns[1:]))

for index,trace in enumerate(data):
    (r, c) = divmod(index, 3)
    fig.append_trace(trace, r+1, c+1)

fig['layout']['yaxis1'].update(title='Ratio (in %)')
fig['layout']['yaxis4'].update(title='Ratio (in %)')
fig['layout']['yaxis7'].update(title='Ratio (in %)')

fig['layout'].update(title='Food Category distribution within the 10 biggest food brands in France.', height=900, width=1000)

py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]
[ (2,1) x4,y4 ]  [ (2,2) x5,y5 ]  [ (2,3) x6,y6 ]
[ (3,1) x7,y7 ]  [ (3,2) x8,y8 ]  [ (3,3) x9,y9 ]



The analysis is hard since for all the brands except 'Picard' and 'Belle France' the products whose category is 'Unknown' represent more than 1/4 of the total number of products of the brand. For 'Picard' and 'Belle France' more than 80% of the products have a relevant tag 'categroy' (e.g 'Beverages'). We stick the analysis to those two brands:   

- we osberve that the brand 'Picard' mainly sells 'Composite food' which is frozen food, which are mainly tagged 'a'. Picard can be called a "healthy brand". 


- on the opposite the brand 'Belle France' has a high ratio (15%) of products tagged 'Sugary Snacks'. It is the most unhealthy food category with a large majority of products tagged 'e' or 'd' regarding their nutrition grade. 'Belle France' sells a lot of "junk" food. However, the food category ratios are more homogeneous than the ones of the brand 'Picard'.      


## Nutrition grade repartition per category in France

In [25]:
# read the data 
cat_nut_df = bra_cat_nut_df[['pnns_groups_1', 'nutrition_grade_fr']]

In [26]:
# number of products (total)
cat_nut_df.index[-1]

223388

In [27]:
# we drop the null value
cat_nut_df = cat_nut_df.dropna()

In [28]:
# number of products (after removing the null)
cat_nut_df.reset_index(drop=True).index[-1]

103469

We removed half the products but having still more than 100,000 products we assume the analysis is still statistically significant.

In [29]:
# we get dummies from the nutrition grades
cat_nut_df = pd.get_dummies(cat_nut_df.set_index('pnns_groups_1')).rename(columns=
                              {'nutrition_grade_fr_a':'a',
                               'nutrition_grade_fr_b':'b',
                               'nutrition_grade_fr_c':'c',
                               'nutrition_grade_fr_d':'d',
                               'nutrition_grade_fr_e':'e'})

In [30]:
# we count the number of products with grade 'a','b' ect. for each food category
count = cat_nut_df.groupby('pnns_groups_1').sum()
count['total'] = count.apply(lambda x: x['a']+x['b']+x['c']+x['d']+x['e'], axis=1) 
count.head(3)

,a,b,c,d,e,total
pnns_groups_1,,,,,,
Beverages,71.0,854.0,1812.0,1624.0,3754.0,8115.0
Cereals And Potatoes,3839.0,1630.0,1616.0,1045.0,101.0,8231.0
Composite Foods,1798.0,3407.0,3404.0,2073.0,185.0,10867.0


In [31]:
# we extract ratio from the counts and the total number of products for each category
ratio = count.copy()
ratio[['a','b','c','d','e']] = count[['a','b','c','d','e']].div(count['total'].values,axis=0)*100
ratio.head(3)

,a,b,c,d,e,total
pnns_groups_1,,,,,,
Beverages,0.874923,10.523722,22.329020,20.012323,46.260012,8115.0
Cereals And Potatoes,46.640748,19.803183,19.633094,12.695906,1.227068,8231.0
Composite Foods,16.545505,31.351799,31.324193,19.076102,1.702402,10867.0


In [32]:
# we remove the 'total' column and we sort the category w.r.t to the ratio for the score 'a' (the healthier)
ratio_to_plot = ratio.reset_index().drop('total',1).sort_values(by=['a'],ascending=0)

# drop 'Unknown' category since not relevant for analysis (regroup multiple categories of products)
ratio_to_plot = ratio_to_plot[ratio_to_plot.pnns_groups_1 != 'Unknown'].reset_index(drop=True)

In [33]:
# interative plot
data = []
colors = sns.color_palette("RdBu_r", 5).as_hex()

for index, nutri in enumerate(ratio_to_plot.columns[1:]):
    data.append(go.Bar(x=ratio_to_plot['pnns_groups_1'],
                       y=ratio_to_plot[ratio_to_plot.columns[index+1]],
                       name = ratio_to_plot.columns[index+1],
                       marker={'color': colors[index]}))

layout = go.Layout(
    barmode='stack',
    title='Health profile for each food category.',
    xaxis=go.layout.XAxis(
        title='Food Category',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        tickangle = 20,
        tickvals=np.arange(10).astype(str),
        ticktext=ratio_to_plot['pnns_groups_1'],
    ),
    yaxis=dict(
        title='Ratio (in %)',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
)

fig = go.FigureWidget(data=data, layout=layout)

py.iplot(fig, filename='food_category_vs_nutriscore', auto_open=False)

As excepted we can obsevrve than the most healthy products are fruits and vegetables... And that the worst are snacks and beverages. 

Interestingly, the sugary snacks are worst for health than the salty snacks or the fatty products. Indeed the ratio of 'e' products is 49% for sugary snacks and 14% for salty snacks or 22% for fat and sauces. Moreover, the sum of 'e'+'d' ratios is also higher for sugary snacks. This could demonstrate that the real ennemy is maybe more 'sugar' than 'salt' or 'fat' which contrast with the standard nutritional recommendations which usually emphasize on fat.

Finally the most heterogeneous category is the 'Fish Meat Eggs' one. It probably contains good products like fish without any industrial transformation and bad products like breaded fish or bacon.

# Brand vs Palm Oil in France

## load and clean

# Load cleaned data

In [34]:
bra_palm_df = pd.read_pickle('bra_palm_cleaned_df')

## Palm oil presence in the whole dataset

In [35]:
bra_palm_bool_df = bra_palm_df.copy()
bra_palm_bool_df.iloc[:,0] = bra_palm_bool_df.iloc[:,0].apply(lambda x: x>0) # for 'ingredients_from_palm_oil_n'
bra_palm_bool_df.iloc[:,1] = bra_palm_bool_df.iloc[:,1].apply(lambda x: x>0) # for 'ingredients_that_may_be_from_palm_oil_n'

In [36]:
bra_palm_to_plot_df = bra_palm_df[['brands']].copy()
bra_palm_to_plot_df['no_palm_oil'] = bra_palm_bool_df.apply(lambda x: 1 if not x[0] and not x[1] else 0, axis=1) 
bra_palm_to_plot_df['may_contain_palm_oil'] = bra_palm_bool_df.apply(lambda x: 1 if x[0] and not x[1] else 0 , axis=1) 
bra_palm_to_plot_df['contain_palm_oil'] = bra_palm_bool_df.apply(lambda x: 1 if x[1] else 0, axis=1) 

In [37]:
bra_palm_to_plot_df.head(3)

,brands,no_palm_oil,may_contain_palm_oil,contain_palm_oil
0,Crous,1,0,0
1,None,1,0,0
2,None,1,0,0


In [38]:
# count the number of product for each palm category 
count = bra_palm_to_plot_df.sum(axis=0)

In [39]:
# interactive plot
data = []
colors = sns.color_palette("Reds", 3).as_hex()
labels = ['no palm oil', 'may contain palm oil', 'contain palm oil']

for index, item in enumerate(count.iteritems()):
    data.append(go.Bar(x=np.array(index),
                       y=np.array(item[1]),
                       name = labels[index],
                       marker={'color': colors[index]}))

layout = go.Layout(
    title='Nutrition score occurences in the original dataset.',
    showlegend=False,
    xaxis=go.layout.XAxis(
        title='Product feature',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        tickvals=np.arange(3).astype(str),
        ticktext= labels,
    ),
    yaxis=dict(
        title='Count',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
)

fig = go.FigureWidget(data=data, layout=layout)
fig

FigureWidget({
    'data': [{'marker': {'color': '#fcbba1'},
              'name': 'no palm oil',
              'type': 'bar',
              'uid': 'fd1f74ac-011c-11e9-8491-ac220bb8d9d3',
              'x': array([0]),
              'y': array([150081])},
             {'marker': {'color': '#fb694a'},
              'name': 'may contain palm oil',
              'type': 'bar',
              'uid': 'fd1f74ad-011c-11e9-89c7-ac220bb8d9d3',
              'x': array([1]),
              'y': array([6611])},
             {'marker': {'color': '#ca181d'},
              'name': 'contain palm oil',
              'type': 'bar',
              'uid': 'fd1f74ae-011c-11e9-8992-ac220bb8d9d3',
              'x': array([2]),
              'y': array([15777])}],
    'layout': {'showlegend': False,
               'title': 'Nutrition score occurences in the original dataset.',
               'xaxis': {'ticktext': [no palm oil, may contain palm oil, contain
                                      palm oil],
                         'tickvals': array(['0', '1', '2'], dtype=object),
                         'title': 'Product feature',
                         'titlefont': {'color': '#7f7f7f', 'family': 'Courier New, monospace', 'size': 18}},
               'yaxis': {'title': 'Count',
                         'titlefont': {'color': '#7f7f7f', 'family': 'Courier New, monospace', 'size': 18}}}
})

In [40]:
# interactive plot
fig = {
    'data': [{'labels': labels,
              'values': count.values,
              'type': 'pie',
              'marker':{'colors':colors}
              }
            ],
    'layout': {'title': 'Palm oil in french products.'}
     }

py.iplot(fig, filename='palm_oil_in_france', auto_open=False)

We observe that the large majority (87%) of the ~170,000 products in the dataset don't contain palm oil. 

However it is important to bear in mind that it is quite hard to (rapidely) assess if one ingredient contains palm oil in case it is not directly mentionned in the ingredients list. Indeed it is unlikely that all the persons who have added a product in the database have done an exhaustive research on the fabrication process of all the ingredients. 

## Palm oil presence for each french food brand

In [41]:
# read the data
bra_palm_df = pd.read_pickle('bra_palm_cleaned_df')

In [42]:
# count number of products per brand
brand_count = bra_palm_df[['ingredients_that_may_be_from_palm_oil_n', 'brands']] \
                                    .groupby('brands')\
                                    .count() \
                                    .rename(columns={'ingredients_that_may_be_from_palm_oil_n':'total'})
brand_count.head(3)

,total
brands,
123 bio,592
7Up,28
A l'olivier,103


In [43]:
# count the number of products for each palm category w.r.t the brands
bra_palm_grouped_df = bra_palm_df.groupby('brands').sum()

# add the number of products per brand
bra_palm_grouped_df = brand_count.join(bra_palm_grouped_df)

bra_palm_grouped_df.head(3)

,total,ingredients_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_n
brands,,,
123 bio,592,2,1
7Up,28,0,0
A l'olivier,103,1,2


In [44]:
# get ratio from the counts
ratio_df = pd.DataFrame()
ratio_df['contain palm oil'] = bra_palm_grouped_df.ingredients_from_palm_oil_n.div(bra_palm_grouped_df.total)
ratio_df['may contain palm oil'] = bra_palm_grouped_df.ingredients_that_may_be_from_palm_oil_n.div(bra_palm_grouped_df.total)
ratio_df['no palm oil'] = 1 - ratio_df['may contain palm oil'] - ratio_df['contain palm oil']

# get percentage
ratio_df = ratio_df.applymap(lambda x: 100*x) 

ratio_df.head(3)

,contain palm oil,may contain palm oil,no palm oil
brands,,,
123 bio,0.337838,0.168919,99.493243
7Up,0.000000,0.000000,100.000000
A l'olivier,0.970874,1.941748,97.087379


In [45]:
# extract the values for the 10 biggest brands in France
ratio_top10_df = ratio_df.copy().reset_index()[ratio_df.index.isin(top10_brand_names)].reset_index(drop=True)

In [46]:
# sort them to have the more "environmental" brands first
ratio_top10_df.sort_values(by='no palm oil', ascending=False, inplace=True)

In [47]:
# interactive plot
data = []
colors = sns.color_palette("tab10", 10).as_hex()

for index, row in ratio_top10_df.iterrows():
    data.append(go.Bar(x=ratio_top10_df.columns[1:],
                       y=np.array(row[1:]),
                       name = row['brands'],
                       marker={'color': colors[index]}))

layout = go.Layout(
    title='Raio of products with palm oil for the top 10 french brand.',
    xaxis=go.layout.XAxis(
        title='Product Feature',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        tickvals=np.arange(3).astype(str),
        ticktext=ratio_top10_df.columns[1:],
    ),
    yaxis=dict(
        title='Ratio (in %)',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
)

go.FigureWidget(data=data, layout=layout)

FigureWidget({
    'data': [{'marker': {'color': '#17becf'},
              'name': 'Picard',
              'type': 'bar',
              'uid': '03238876-011d-11e9-8816-ac220bb8d9d3',
              'x': array(['contain palm oil', 'may contain palm oil', 'no palm oil'], dtype=object),
              'y': array([1.6371971185330716, 13.883431565160445, 84.47937131630648], dtype=object)},
             {'marker': {'color': '#7f7f7f'},
              'name': 'Monoprix',
              'type': 'bar',
              'uid': '03238877-011d-11e9-b2b0-ac220bb8d9d3',
              'x': array(['contain palm oil', 'may contain palm oil', 'no palm oil'], dtype=object),
              'y': array([1.7937219730941705, 14.221652786675207, 83.98462524023061], dtype=object)},
             {'marker': {'color': '#d62728'},
              'name': 'Casino',
              'type': 'bar',
              'uid': '03238878-011d-11e9-87cf-ac220bb8d9d3',
              'x': array(['contain palm oil', 'may contain palm oil', 'no palm oil'], dtype=object),
              'y': array([3.10254735467015, 14.500326583932072, 82.39712606139777], dtype=object)},
             {'marker': {'color': '#2ca02c'},
              'name': 'Carrefour',
              'type': 'bar',
              'uid': '03238879-011d-11e9-ae3b-ac220bb8d9d3',
              'x': array(['contain palm oil', 'may contain palm oil', 'no palm oil'], dtype=object),
              'y': array([5.850263355906696, 12.584650112866816, 81.5650865312265], dtype=object)},
             {'marker': {'color': '#9467bd'},
              'name': 'Cora',
              'type': 'bar',
              'uid': '0323887a-011d-11e9-b7eb-ac220bb8d9d3',
              'x': array(['contain palm oil', 'may contain palm oil', 'no palm oil'], dtype=object),
              'y': array([6.375502008032129, 13.303212851405622, 80.32128514056225], dtype=object)},
             {'marker': {'color': '#1f77b4'},
              'name': 'Auchan',
              'type': 'bar',
              'uid': '0323887b-011d-11e9-b3f7-ac220bb8d9d3',
              'x': array(['contain palm oil', 'may contain palm oil', 'no palm oil'], dtype=object),
              'y': array([5.241460541813899, 15.213977228111503, 79.54456223007459], dtype=object)},
             {'marker': {'color': '#8c564b'},
              'name': 'Leader Price',
              'type': 'bar',
              'uid': '0323887c-011d-11e9-8e6a-ac220bb8d9d3',
              'x': array(['contain palm oil', 'may contain palm oil', 'no palm oil'], dtype=object),
              'y': array([5.447470817120623, 17.438981252210827, 77.11354793066855], dtype=object)},
             {'marker': {'color': '#bcbd22'},
              'name': 'Netto',
              'type': 'bar',
              'uid': '0323887d-011d-11e9-a588-ac220bb8d9d3',
              'x': array(['contain palm oil', 'may contain palm oil', 'no palm oil'], dtype=object),
              'y': array([6.967213114754098, 16.311475409836067, 76.72131147540983], dtype=object)},
             {'marker': {'color': '#ff7f0e'},
              'name': 'Belle France',
              'type': 'bar',
              'uid': '0323887e-011d-11e9-aabb-ac220bb8d9d3',
              'x': array(['contain palm oil', 'may contain palm oil', 'no palm oil'], dtype=object),
              'y': array([6.55893536121673, 16.82509505703422, 76.61596958174904], dtype=object)},
             {'marker': {'color': '#e377c2'},
              'name': 'Leclerc',
              'type': 'bar',
              'uid': '0323887f-011d-11e9-a6c3-ac220bb8d9d3',
              'x': array(['contain palm oil', 'may contain palm oil', 'no palm oil'], dtype=object),
              'y': array([7.310302282541641, 20.913016656384947, 71.77668106107342], dtype=object)}],
    'layout': {'title': 'Raio of products with palm oil for the top 10 french brand.',
               'xaxis': {'ticktext': array(['contain palm oil', 'may contain palm oil', 'no palm oil'], dtype=object),
                         'tickvals': array(['0', '1', '2'], dtype

In [48]:
# interactive plot
data = []
colors = ['#8B0000', '#FFA500', '#008000']

for i in range(3):
    
    data.append(go.Bar(x=ratio_top10_df['brands'],
                       y=ratio_top10_df[ratio_top10_df.columns[i+1]],
                       name = ratio_top10_df.columns[i+1],
                       marker={'color': colors[i]}))

layout = go.Layout(
    title='Palm oil presence within the 10 biggest food brands in France.',
    barmode='stack',
    xaxis=go.layout.XAxis(
        title='Food Brand',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        tickvals=np.arange(10).astype(str),
        ticktext=ratio_top10_df['brands'],
    ),
    yaxis=dict(
        title='Ratio (in %)',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
)

fig = go.FigureWidget(data=data, layout=layout)
py.iplot(fig, filename='palm_oil_vs_brands', auto_open=False)

For all the brands more than 70% of the products does NOT contain palm oil.  

- However, 27% of the products made by 'Leclerc' contain or may contain palm oil, which is a high ratio.
- 'Picard' and  'Monoprix' are the more "environmental" brand regarding palm oil presence in their products.

# Score each brand from nutrition score ratios

In [49]:
brand_nutri_ratio.head(3)

,a,b,c,d,e,total
brands,,,,,,
Carrefour,19.763931,15.344496,19.818831,26.269558,18.803184,3643
Auchan,18.391764,15.303283,21.313300,26.794658,18.196995,3594
Leclerc,13.648180,15.901213,22.010399,27.816291,20.623917,2308


In [50]:
# keep only the brand which have more than 100 products
brand_nutri_ratio = brand_nutri_ratio[brand_nutri_ratio.total > 100]

In [51]:
# we will assign a score to each brand. The best score is 5 (for 'a') and the worst is 1 ('e'). 
scores = np.array([5,4,3,2,1])

# for each brand the score will be the average of the ratios in each category ('a','b'..) weighted by the score of each category (5,4..) 
brand_score_nutri = brand_nutri_ratio.drop('total', axis=1).multiply(1/100).multiply(scores).mean(axis=1).to_frame(name='nutrition score')
brand_score_nutri['nutrition score'] *= 5
brand_score_nutri.sort_values(by='nutrition score', ascending=False, inplace=True)

A score of 5 for a brand means that 100% of the products sold by the brand are 'a' (very healthy). 

In [52]:
# take the 10 best (highest score) and 10 worst brand (lowest score)
brand_10best_10worst = pd.concat([brand_score_nutri.head(10).reset_index(), brand_score_nutri.tail(10).reset_index()], axis=0).reset_index(drop=True)

In [53]:
# interactive plot
data = []
colors = sns.color_palette("RdBu_r", 20).as_hex()
y_values = ['a','b','c','d','e']

for index, row in brand_10best_10worst.iterrows():
    data.append(go.Bar(x=np.array(index),
                       y=np.array(row['nutrition score']),
                       name = row['brands'],
                       marker={'color': colors[index]}))

layout = go.Layout(
    showlegend=False,
    title='Health score of the 10 best and 10 worst food brands in France.',
    xaxis=go.layout.XAxis(
        title='Brand',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        tickvals=np.arange(20).astype(str),
        ticktext=brand_10best_10worst.brands.values,
    ),
    yaxis=dict(
        title='Global Nutrition Score',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        ),
        showgrid = True,
        gridcolor='#bdbdbd',
        range=[0, 6],
        tickwidth=3,
        tickvals=np.arange(1,6).astype(str),
        ticktext=y_values[::-1],
    ),
)

fig = go.FigureWidget(data=data, layout=layout)
py.iplot(fig, filename='mean_nutriscore_vs_brand', auto_open=False)

From this bar plot we can the best and the worst brands for health. 

- As we could expect, eating chocolate is not good... But, suprisingly the 6 worst brands are all chocolate ones except Monin which sell sirop. 

- The best brands are 'D'Aucy' - which sells a lot of vegetables and fruits in cans - and 'Charles et Alice' - which sells fruits in cup.  

In [54]:
brand_of_interest = brand_dummies_count[brand_dummies_count.index.isin(["D'Aucy", "Charles & Alice"])]
brand_of_interest

,Beverages,Cereals And Potatoes,Composite Foods,Fat And Sauces,Fish Meat Eggs,Fruits And Vegetables,Milk And Dairy Products,Salty Snacks,Sugary Snacks,Unknown,total
brands,,,,,,,,,,,
Charles & Alice,2.0,1.0,1.0,1.0,0.0,83.0,9.0,0.0,4.0,56.0,157.0
D'Aucy,0.0,4.0,49.0,0.0,0.0,94.0,0.0,1.0,0.0,51.0,199.0
